# Linear Programming with PuLP: optimizing a diet

I read this [article](https://towardsdatascience.com/linear-programming-and-discrete-optimization-with-python-using-pulp-449f3c5f6e99) and wanted to give it a try myself.

The idea is that we run a cantine. We have food items, and for each item we know the price and the nutrients.
We want to optimize the price, staying within the boundaries of ninimum and maximum daily intake of the nutrients.

## Data

I found the github project for that article.
It has the source [data](https://github.com/tirthajyoti/Optimization-Python), including the `diet-medium.xls` file
(I removed the spaces in the filename and in a food name).

We use pandas to read the excel.

In [1]:
import pandas as pd

The file has 17 data rows, after that there is some "rubbish", so we explcitly limit the number fo rows to read.

In [2]:
df = pd.read_excel("diet-medium.xls",nrows=17)

Inspect the data frame

In [3]:
df

,Foods,Price/Serving,Serving Size,Calories,Cholesterol (mg),Total_Fat (g),Sodium (mg),Carbohydrates (g),Dietary_Fiber (g),Protein (g),Vit_A (IU),Vit_C (IU),Calcium (mg),Iron (mg)
0,Frozen Broccoli,0.48,10 Oz Pkg,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,Frozen Corn,0.54,1/2 Cup,72.2,0.0,0.6,2.5,17.1,2.0,2.5,106.6,5.2,3.3,0.3
2,Raw Lettuce Iceberg,0.06,1 Leaf,2.6,0.0,0.0,1.8,0.4,0.3,0.2,66.0,0.8,3.8,0.1
3,Baked Potatoes,0.18,1/2 Cup,171.5,0.0,0.2,15.2,39.9,3.2,3.7,0.0,15.6,22.7,4.3
4,Tofu,0.93,1/4 block,88.2,0.0,5.5,8.1,2.2,1.4,9.4,98.6,0.1,121.8,6.2
5,Roasted Chicken,2.52,1 lb chicken,277.4,129.9,10.8,125.6,0.0,0.0,42.2,77.4,0.0,21.9,1.8
6,Spaghetti W/ Sauce,2.34,1 1/2 Cup,358.2,0.0,12.3,1237.1,58.3,11.6,8.2,3055.2,27.9,80.2,2.3
7,Raw Apple,0.72,"1 Fruit,3/Lb,Wo/Rf",81.4,0.0,0.5,0.0,21.0,3.7,0.3,73.1,7.9,9.7,0.2
8,Banana,0.45,"1 Fruit,Wo/Skn&Seeds",104.9,0.0,0.5,1.1,26.7,2.7,1.2,92.3,10.4,6.8,0.4
9,Wheat Bread,0.15,1 Sl,65.0,0.0,1.0,134.5,12.4,1.3,2.2,0.0,0.0,10.8,0.7


Observe that for each "Foods" item we know nutrients like Fat, but also Calories.

This is the list of foods.

In [4]:
foods = list(df['Foods'])
foods

['Frozen Broccoli',
 'Frozen Corn',
 'Raw Lettuce Iceberg',
 'Baked Potatoes',
 'Tofu',
 'Roasted Chicken',
 'Spaghetti W/ Sauce',
 'Raw Apple',
 'Banana',
 'Wheat Bread',
 'White Bread',
 'Oatmeal Cookies',
 'Apple Pie',
 'Scrambled Eggs',
 'Turkey Bologna',
 'Beef Frankfurter',
 'Chocolate Chip Cookies']

We make a `cost` dictionary, mapping Foods to Price. Instead of using `df['Foods']` we could also use the just created `foods`.

In [5]:
costs = dict(zip(df['Foods'],df['Price/Serving']))
costs

{'Frozen Broccoli': 0.48,
 'Frozen Corn': 0.54,
 'Raw Lettuce Iceberg': 0.06,
 'Baked Potatoes': 0.18,
 'Tofu': 0.9299999999999999,
 'Roasted Chicken': 2.52,
 'Spaghetti W/ Sauce': 2.34,
 'Raw Apple': 0.72,
 'Banana': 0.44999999999999996,
 'Wheat Bread': 0.15000000000000002,
 'White Bread': 0.18,
 'Oatmeal Cookies': 0.27,
 'Apple Pie': 0.48,
 'Scrambled Eggs': 0.33,
 'Turkey Bologna': 0.44999999999999996,
 'Beef Frankfurter': 0.81,
 'Chocolate Chip Cookies': 0.09}

Similarly we make a `cals` dictionary mapping Foods to Calories, and for `fats`

In [6]:
cals = dict(zip(df['Foods'],df['Calories']))
cals

{'Frozen Broccoli': 73.8,
 'Frozen Corn': 72.2,
 'Raw Lettuce Iceberg': 2.6,
 'Baked Potatoes': 171.5,
 'Tofu': 88.2,
 'Roasted Chicken': 277.4,
 'Spaghetti W/ Sauce': 358.2,
 'Raw Apple': 81.4,
 'Banana': 104.9,
 'Wheat Bread': 65.0,
 'White Bread': 65.0,
 'Oatmeal Cookies': 81.0,
 'Apple Pie': 67.2,
 'Scrambled Eggs': 99.6,
 'Turkey Bologna': 56.4,
 'Beef Frankfurter': 141.8,
 'Chocolate Chip Cookies': 78.1}

In [7]:
fats = dict(zip(df['Foods'],df['Total_Fat (g)']))
fats

{'Frozen Broccoli': 0.8,
 'Frozen Corn': 0.6,
 'Raw Lettuce Iceberg': 0.0,
 'Baked Potatoes': 0.2,
 'Tofu': 5.5,
 'Roasted Chicken': 10.8,
 'Spaghetti W/ Sauce': 12.3,
 'Raw Apple': 0.5,
 'Banana': 0.5,
 'Wheat Bread': 1.0,
 'White Bread': 1.0,
 'Oatmeal Cookies': 3.3,
 'Apple Pie': 3.1,
 'Scrambled Eggs': 7.3,
 'Turkey Bologna': 4.3,
 'Beef Frankfurter': 12.8,
 'Chocolate Chip Cookies': 4.5}

## Setting up the problem

We import pulp, and create variables, one for each `foods` item. That variable holds the quantity for that foods items, hence it is non-integer ("continuous"), but also non-negative (lower bound 0).

In [8]:
import pulp

In [9]:
vars = pulp.LpVariable.dicts("var",df['Foods'],lowBound=0,cat='Continuous')
vars

{'Frozen Broccoli': var_Frozen_Broccoli,
 'Frozen Corn': var_Frozen_Corn,
 'Raw Lettuce Iceberg': var_Raw_Lettuce_Iceberg,
 'Baked Potatoes': var_Baked_Potatoes,
 'Tofu': var_Tofu,
 'Roasted Chicken': var_Roasted_Chicken,
 'Spaghetti W/ Sauce': var_Spaghetti_W__Sauce,
 'Raw Apple': var_Raw_Apple,
 'Banana': var_Banana,
 'Wheat Bread': var_Wheat_Bread,
 'White Bread': var_White_Bread,
 'Oatmeal Cookies': var_Oatmeal_Cookies,
 'Apple Pie': var_Apple_Pie,
 'Scrambled Eggs': var_Scrambled_Eggs,
 'Turkey Bologna': var_Turkey_Bologna,
 'Beef Frankfurter': var_Beef_Frankfurter,
 'Chocolate Chip Cookies': var_Chocolate_Chip_Cookies}

We will now create the linear programming problem; it will aim at minimizing the cost of the diet.

In [10]:
prob = pulp.LpProblem("diet",pulp.LpMinimize)

The next step feels strange to me: we "add" the objective function, the cost in our case.

In [11]:
prob += pulp.lpSum( [costs[f]*vars[f] for f in foods] ), "Cost of the diet"

The spreadsheet has, for each ingredient a minimum and maximum value. For example, for Callories it lists 800 respectively 1300. We again "add" those constrains to our problem.

In [12]:
prob += pulp.lpSum( [cals[f] * vars[f] for f in foods] ) >= 800.0, "Minimum calories"
prob += pulp.lpSum( [cals[f] * vars[f] for f in foods] ) <= 1300.0, "Maximum calories"

We do the same for fat (excel shows 20 and 50 gram as limits).

In [13]:
prob += pulp.lpSum( [fats[f] * vars[f] for f in foods] ) >= 20.0, "Minimum fat"
prob += pulp.lpSum( [fats[f] * vars[f] for f in foods] ) <= 50.0, "Maximum fat"

We save the problem to a file.

In [14]:
# The problem data is written to an .lp file
prob.writeLP("diet.lp");

The content of that file is very readble.

In [15]:
!type diet.lp

\* diet *\
Minimize
Cost_of_the_diet: 0.48 var_Apple_Pie + 0.18 var_Baked_Potatoes
 + 0.45 var_Banana + 0.81 var_Beef_Frankfurter
 + 0.09 var_Chocolate_Chip_Cookies + 0.48 var_Frozen_Broccoli
 + 0.54 var_Frozen_Corn + 0.27 var_Oatmeal_Cookies + 0.72 var_Raw_Apple
 + 0.06 var_Raw_Lettuce_Iceberg + 2.52 var_Roasted_Chicken
 + 0.33 var_Scrambled_Eggs + 2.34 var_Spaghetti_W__Sauce + 0.93 var_Tofu
 + 0.45 var_Turkey_Bologna + 0.15 var_Wheat_Bread + 0.18 var_White_Bread
Subject To
Maximum_calories: 67.2 var_Apple_Pie + 171.5 var_Baked_Potatoes
 + 104.9 var_Banana + 141.8 var_Beef_Frankfurter
 + 78.1 var_Chocolate_Chip_Cookies + 73.8 var_Frozen_Broccoli
 + 72.2 var_Frozen_Corn + 81 var_Oatmeal_Cookies + 81.4 var_Raw_Apple
 + 2.6 var_Raw_Lettuce_Iceberg + 277.4 var_Roasted_Chicken
 + 99.6 var_Scrambled_Eggs + 358.2 var_Spaghetti_W__Sauce + 88.2 var_Tofu
 + 56.4 var_Turkey_Bologna + 65 var_Wheat_Bread + 65 var_White_Bread <= 1300
Maximum_fat: 3.1 var_Apple_Pie + 0.2 var_Baked_Potatoes + 0.5 var

## Solving the problem

When we try to solve it, we find an "Optimal" solution.

In [16]:
stat = prob.solve()
pulp.LpStatus[stat]

'Optimal'

Which are the components of the cost optimized meal (with cals and fats constraints)?

In [17]:
for v in prob.variables() : 
    if v.varValue>0.0 : 
        print( f"{v.name}={v.varValue}" )

var_Baked_Potatoes=2.6953037
var_Chocolate_Chip_Cookies=4.3246532


Doesn't look too healthy, but it is dirt-cheap:

In [18]:
pulp.value(prob.objective)

0.874373454

Looking up the price per portion from the excel, we can check that; it matches.

In [19]:
2.695*0.18 + 4.324*0.09

0.8742599999999999

## Improving the diet

Let's add fiber to the constraints.

In [20]:
fiber = dict(zip(df['Foods'],df['Dietary_Fiber (g)']))
prob += pulp.lpSum( [fiber[f] * vars[f] for f in foods] ) >= 60.0, "Minimum fiber"
prob += pulp.lpSum( [fiber[f] * vars[f] for f in foods] ) <= 125.0, "Maximum fiber"

Solve the problem again

In [21]:
stat = prob.solve()
pulp.LpStatus[stat]

'Optimal'

Print the meal components

In [22]:
for v in prob.variables() : 
    if v.varValue>0.0 : 
        print( f"{v.name:26} = {v.varValue}" )

var_Baked_Potatoes         = 0.2059191
var_Chocolate_Chip_Cookies = 3.1941723
var_Frozen_Broccoli        = 6.981301


and the price. Broccoli is in, and the price is up!

In [23]:
c = pulp.value(prob.objective)
print( f"${c:.2f}" )

$3.68


(end)